# EDA weather data transformations

In [0]:
#imports
import matplotlib.pyplot as plt
import time
import numpy as np 
import pandas as pd
import time
import numpy as np 
import pandas as pd
import seaborn as sns
from pytz import timezone 
from datetime import  datetime, timedelta 

blob_container = "blobcontainer" # The name of your container created in https://portal.azure.com
storage_account = "w261section05group03" # The name of your Storage account created in https://portal.azure.com
secret_scope = "w261section05group04" # The name of the scope created in your local computer using the Databricks CLI
secret_key = "houseofthedragon" # The name of the secret key created in your local computer using the Databricks CLI 
blob_url = f"wasbs://{blob_container}@{storage_account}.blob.core.windows.net"
mount_path = "/mnt/mids-w261"

spark.conf.set(
  f"fs.azure.sas.{blob_container}.{storage_account}.blob.core.windows.net",
  dbutils.secrets.get(scope = secret_scope, key = secret_key)
)

data_BASE_DIR = "dbfs:/mnt/mids-w261/datasets_final_project_2022/"
display(dbutils.fs.ls(f"{data_BASE_DIR}stations_data/"))

path,name,size,modificationTime
dbfs:/mnt/mids-w261/datasets_final_project_2022/stations_data/stations_with_neighbors.parquet/,stations_with_neighbors.parquet/,0,1656713664000


In [0]:
#save transformed weather data----
#recover the file from the blob
#test.write.mode("overwrite").parquet(f"{blob_url}/df_us_weather_modJR")
df_us_weather = spark.read.parquet(f"{blob_url}/df_us_weather_modJR")
#display(df_us_weather2)

In [0]:
cols_to_drop = [ 'Sunrise'
 'Sunset',
 'DailyAverageDewPointTemperature',
 'DailyAverageDryBulbTemperature',
 'DailyAverageRelativeHumidity',
 'DailyAverageSeaLevelPressure',
 'DailyAverageStationPressure',
 'DailyAverageWetBulbTemperature',
 'DailyAverageWindSpeed',
 'DailyCoolingDegreeDays',
 'DailyDepartureFromNormalAverageTemperature',
 'DailyHeatingDegreeDays',
 'DailyMaximumDryBulbTemperature'
 'DailyMinimumDryBulbTemperature'
 'DailyPeakWindDirection',
 'DailyPeakWindSpeed',
 'DailyPrecipitation',
 'DailySnowDepth',
 'DailySnowfall',
 'DailySustainedWindDirection',
 'DailySustainedWindSpeed',
 'DailyWeather',
'Sunrise',
'Sunset',
'DailyMaximumDryBulbTemperature',
'DailyMinimumDryBulbTemperature',
'DailyPeakWindDirection',
'origin',
'ident',
'neighbor_id',
'min(distance_to_neighbor)',
'count(DATE)',
'City',
'Country',
'dayofweek',
'weekday',
'month',
'hour',
'TIMESTAMP',
'date_use',
'State','REM',
'HourlyPressureChange',
'HourlyPressureTendency',
'HourlyRelativeHumidity',
'HourlySkyConditions',
'HourlySeaLevelPressure',
'HourlyAltimeterSetting',
'HourlyWindGustSpeed',
'HourlyAltimeterSetting']


test = df_us_weather.drop(*cols_to_drop)

test.printSchema()

root
 |-- STATION: string (nullable = true)
 |-- DATE: string (nullable = true)
 |-- LATITUDE: string (nullable = true)
 |-- LONGITUDE: string (nullable = true)
 |-- ELEVATION: string (nullable = true)
 |-- NAME: string (nullable = true)
 |-- REPORT_TYPE: string (nullable = true)
 |-- SOURCE: string (nullable = true)
 |-- HourlyDewPointTemperature: string (nullable = true)
 |-- HourlyDryBulbTemperature: string (nullable = true)
 |-- HourlyPrecipitation: string (nullable = true)
 |-- HourlyPresentWeatherType: string (nullable = true)
 |-- HourlyStationPressure: string (nullable = true)
 |-- HourlyVisibility: string (nullable = true)
 |-- HourlyWetBulbTemperature: string (nullable = true)
 |-- HourlyWindDirection: string (nullable = true)
 |-- HourlyWindSpeed: string (nullable = true)
 |-- YEAR: integer (nullable = true)



In [0]:
#calculate number of missing records

from pyspark.sql.functions import col,isnan,when,count,lit
df_Columns= test
null_df = df_Columns.select([(count(when(isnan(c) | col(c).isNull(), c))*100./count(lit(1))).alias(c) for c in df_Columns.columns]).toPandas()
null_df2 = df_Columns.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_Columns.columns]).toPandas()
                             
null_df = null_df.transpose()
null_df2 = null_df2.transpose()
null_df.rename(columns = {0:"% of Total Values"}, inplace = True)
null_df2.rename(columns = {0:"Missing Values"}, inplace = True)

df_missing = pd.concat([null_df2,null_df],axis = 1)
df_missing.sort_values(by ='% of Total Values', ascending=False )

,Missing Values,% of Total Values
HourlyPresentWeatherType,24515707,77.752286
HourlyPrecipitation,8651007,27.436923
HourlyWetBulbTemperature,805892,2.555910
HourlyStationPressure,708933,2.248402
HourlyDewPointTemperature,124438,0.394659
HourlyWindDirection,112043,0.355348
HourlyWindSpeed,110942,0.351856
HourlyDryBulbTemperature,76459,0.242492
HourlyVisibility,54697,0.173473
STATION,0,0.000000


In [0]:
df_us_weather=test 

### Feature: hourly precipitation

None and T values are set to zero.

In [0]:
#clean values oh hourly precipitation
def clean_hourly_precipitation(line, value= 0.0):
  if (line is None):
    value = 0.0
  elif ('T' in line) | (line == "") | ('*' in line):
    value = 0.0
  elif 's' in line:
    temp = line.replace('s', '')
    if len(line)>=8:
      value = (float(temp[:4])+float(temp[4:])) /2.0
    else:
      value = float(temp)
  elif len(line)>=8:
      value = (float(line[:4])+float(line[4:])) /2.0     
  else:
    value = float(line)
  return value  

clean_hourly_precipitation = udf(clean_hourly_precipitation)
# Create the timestamp for the weather data
df_us_weather = df_us_weather.withColumn("HourlyPrecipitation", clean_hourly_precipitation('HourlyPrecipitation'))

###Feature: hourly temperatures

cast as numeric.

In [0]:
# converte the hourly temperatures to numeric
from pyspark.sql.functions import col
columns_to_cast = ["HourlyDewPointTemperature","HourlyDryBulbTemperature","HourlyWetBulbTemperature"]
df_us_weather = (
   df_us_weather
   .select(
     *(c for c in df_us_weather.columns if c not in columns_to_cast),
     *(col(c).cast("float").alias(c) for c in columns_to_cast)
   )
)

### Hourly Pressure  and tendency

Caset Hourly pressure

In [0]:
# converte the hourly Station Pressure to numeric
from pyspark.sql.functions import col

columns_to_cast = ["HourlyStationPressure"]
df_us_weather = (
   df_us_weather
   .select(
     *(c for c in df_us_weather.columns if c not in columns_to_cast),
     *(col(c).cast("float").alias(c) for c in columns_to_cast)
   )
)

###  HourlyWindDirection    HourlyWindSpeed

Casto numeric, convert NaNs  or missing to zero.

In [0]:
# converte the hourly  to numeric
from pyspark.sql.functions import col

columns_to_cast = ["HourlyWindDirection", "HourlyWindSpeed"]
df_us_weather = (
   df_us_weather
   .select(
     *(c for c in df_us_weather.columns if c not in columns_to_cast),
     *(col(c).cast("float").alias(c) for c in columns_to_cast)
   )
)

### Visiblity

cast to numeric replace values above 30 with 30

In [0]:
# converte the hourly  to numeric
from pyspark.sql.functions import col

columns_to_cast = ["HourlyVisibility"]
df_us_weather = (
   df_us_weather
   .select(
     *(c for c in df_us_weather.columns if c not in columns_to_cast),
     *(col(c).cast("float").alias(c) for c in columns_to_cast)
   )
)

### Category of weather

This feature reports precipitaitno or obstructions ocurring at time of observation.

- Thunderstorm / Lightning: 26, 90-99
- Ice/Freezing Rain: 25, 45-48, 54-56, 64-68
- Snow: 24, 70-78, 85-87
- Fog :  FG:30 FG:35

In [0]:
##Clean the Hourly weather event


from pyspark.sql.functions import col, to_timestamp, monotonically_increasing_id, datediff, when, split

df_us_weather = df_us_weather.withColumn('AU_code', split(col('HourlyPresentWeatherType'), '\\|').getItem(0))\
                               .withColumn('AW_code', split(col('HourlyPresentWeatherType'), '\\|').getItem(1))\
                               .withColumn('MW_code', split(col('HourlyPresentWeatherType'), '\\|').getItem(2))\

df_us_weather = df_us_weather.withColumn('AU_TS', when( (col('AU_code').contains("+TS")) | (col('AU_code').contains("FC")),1).otherwise(0))\
                                   .withColumn('AU_ICE', when(col('AU_code').contains("IC" ), 1).otherwise(0))\
                                   .withColumn('AU_SNOW',when(col('AU_code').contains("+SN" ), 1).otherwise(0))\
                                   .withColumn('AU_FOG', when(col('AU_code').contains("FG" ), 1).otherwise(0))

df_us_weather = df_us_weather.withColumn('AW_TS', when((col('AW_code').contains("TS" ))| (col('AW_code').contains("FC")) , 1).otherwise(0))\
                                   .withColumn('AW_ICE', when(col('AW_code').contains("FZRA" ), 1).otherwise(0))\
                                   .withColumn('AW_SNOW',when(col('AW_code').contains("SN"), 1).otherwise(0))\
                                   .withColumn('AW_FOG', when(col('AW_code').contains("FG"), 1).otherwise(0))

df_us_weather = df_us_weather.withColumn('MW_TS', when((col('MW_code').contains("TS" ))| (col('MW_code').contains("FC")), 1).otherwise(0))\
                                  .withColumn('MW_ICE', when(col('MW_code').contains("FZRA"), 1).otherwise(0))\
                                  .withColumn('MW_SNOW',when(col('MW_code').contains("SN" ), 1).otherwise(0))\
                                  .withColumn('MW_FOG', when(col('MW_code').contains("FG"), 1).otherwise(0))

df_us_weather = df_us_weather.withColumn('TS',when( (col('AU_TS')== 1) | (col('AW_TS')== 1)| (col('MW_TS')== 1), 1).otherwise(0))\
                                  .withColumn('ICE', when((col('AU_ICE')== 1 )| (col('AW_ICE')== 1)| (col('MW_ICE')== 1), 1).otherwise(0))\
                                  .withColumn('SNOW',when((col('AU_SNOW')== 1 )| (col('AW_SNOW')== 1)| (col('MW_SNOW')== 1), 1).otherwise(0))\
                                  .withColumn('FOG', when((col('AU_FOG')== 1 )| (col('AW_FOG')== 1)| (col('MW_FOG')== 1), 1).otherwise(0))\



In [0]:
#calculate number of missing records

from pyspark.sql.functions import col,isnan,when,count,lit
df_Columns= df_us_weather
null_df = df_Columns.select([(count(when(isnan(c) | col(c).isNull(), c))*100./count(lit(1))).alias(c) for c in df_Columns.columns]).toPandas()
null_df2 = df_Columns.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_Columns.columns]).toPandas()
                             
null_df = null_df.transpose()
null_df2 = null_df2.transpose()
null_df.rename(columns = {0:"% of Total Values"}, inplace = True)
null_df2.rename(columns = {0:"Missing Values"}, inplace = True)

df_missing = pd.concat([null_df2,null_df],axis = 1)
df_missing.sort_values(by ='% of Total Values', ascending=False )

,Missing Values,% of Total Values
AU_code,24515707,77.752286
HourlyPresentWeatherType,24515707,77.752286
MW_code,24515707,77.752286
AW_code,24515707,77.752286
HourlyWindDirection,1129215,3.581339
HourlyWetBulbTemperature,831870,2.638300
HourlyStationPressure,714885,2.267279
HourlyVisibility,238113,0.755182
HourlyDewPointTemperature,159097,0.504581
HourlyDryBulbTemperature,122036,0.387041
